In [ ]:
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils.class_weight import compute_class_weight

seed=42
np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# --- Evaluation ---
def evaluate_model(model, name):
    pred = np.argmax(model.predict(X_test, verbose=0), axis=1)
    true = y_test
    acc = accuracy_score(true, pred)
    print(f"{name} Accuracy: {acc:.4f}")
    return acc

# --- Meta-Meta Learner ---
def create_meta_meta_learner(meta_learners, seed=42):
    inputs = tf.keras.Input(shape=input_shape, name="MetaMeta_input")
    meta_outputs = [ml(inputs) for ml in meta_learners]
    stacked = layers.Concatenate()(meta_outputs)
    stacked = layers.RepeatVector(2)(stacked)
    x = layers.LSTM(64, activation='tanh', seed=seed)(concat)
    out = layers.Dense(NUM_CLASSES, activation='softmax')(x)
    model = models.Model(inputs=inputs, outputs=out, name="MetaMetaLearner")
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# --- Meta Learner ---
def create_meta_learner(base_models, name="MetaLearner", seed=42):
    inputs = tf.keras.Input(shape=input_shape, name=f"{name}_input")
    base_outputs = [model(inputs) for model in base_models]
    stacked = layers.Concatenate()(base_outputs)
    stacked = layers.RepeatVector(3)(stacked)
    x = layers.LSTM(64, activation='tanh', seed=seed)(stacked)
    out = layers.Dense(NUM_CLASSES, activation='softmax')(x)
    meta_model = models.Model(inputs=inputs, outputs=out, name=name)
    meta_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return meta_model

# --- Base Model Generator ---
def create_base_model(hidden_units=32, seed=42):
    model = tf.keras.Sequential([
        layers.Input(shape=input_shape),
        layers.LSTM(hidden_units, activation='tanh', seed=seed),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    model.compile(optimizer='adamw', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# --- Train Base Models ---
def train_base_models(num_models=3, label="A"):
    models = []
    for i in range(num_models):
        callback = [
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1),
            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
        ]
        model = create_base_model(hidden_units=32 + i * 16, seed=i)
        print(f"Training BaseModel_{label}{chr(97+i)}...")
        model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_val, y_val),
                  verbose=1, callbacks=callback, class_weight=class_weights_dict)
        models.append(model)
    return models

# --- Get real world data ---
def get_real_data(dataset):
    """Load and preprocess real data from a CSV."""
    print('\nBuilding dataframe using real data...')

    if dataset == 'Euro' or dataset == 'Thunderball' or dataset == 'Take5':
        cols = ['A', 'B', 'C', 'D', 'E']
    else:
        cols = ['A', 'B', 'C', 'D', 'E', 'F']

    if dataset == 'Take5':
        df = pd.read_csv(f'datasets/training/{dataset}_Full.csv').astype(np.int16)
    else:
        df = pd.read_csv(f'datasets/UK/{dataset}_ascend.csv').astype(np.int16)

    df = df[cols].dropna().astype(np.int16)
    # Format each element as 2-digit string and then flatten digits into an array.
    df = df.map(lambda x: f'{x:02d}')
    flattened = df.values.flatten()
    full_data = np.array([int(d) for num in flattened for d in str(num)], dtype=np.int16)
    return full_data

# --- Generate data sequences ---
def generate_dataset(data, wl=10, features=1):
    X_test = np.empty([len(data)-wl, wl, features], dtype=np.int8)
    y_test = np.empty([len(data)-wl, 1], dtype=np.int8)
    for i in range(len(data)-wl):
        X_test[i] = data[i:i+wl]
        y_test[i] = data[i+wl]    
    return X_test, y_test


# --- Parameters ---
NUM_CLASSES = 10
NOISE_LEVEL = 0.1
EPOCHS = 3
BATCH_SIZE = 64
WL = 10
dataset = 'Take5'
wl = 10

X_raw = get_real_data(dataset)
data = X_raw.reshape(-1,1)
features = data.shape[1]
input_shape = (wl, features)

X, y = generate_dataset(data, wl, features)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)

unique_classes = np.unique(y.flatten())
class_weights = compute_class_weight('balanced', classes=unique_classes, y=y.flatten())
class_weights_dict = dict(enumerate(class_weights))

base_models_1 = train_base_models(label="1")
base_models_2 = train_base_models(label="2")

# --- Train Meta Learners ---
meta_learner_1 = create_meta_learner(base_models_1, name="Meta1", seed=4)
meta_learner_2 = create_meta_learner(base_models_2, name="Meta2", seed=5)

print("Training MetaLearner 1...")
callback = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
]
meta_learner_1.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_val, y_val),
                   verbose=1, callbacks=callback, class_weight=class_weights_dict)

print("Training MetaLearner 2...")
callback = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
]
meta_learner_2.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_val, y_val),
                   verbose=1, callbacks=callback, class_weight=class_weights_dict)

# --- Train Meta-Meta Learners ---
meta_meta_learner = create_meta_meta_learner([meta_learner_1, meta_learner_2], seed=6)

print("Training Meta-Meta Learner...")
callback = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
]
meta_meta_learner.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_val, y_val),
                      verbose=1, callbacks=callback, class_weight=class_weights_dict)

print("\n--- Final Evaluation ---")
subs = ['a', 'b', 'c']
for i, sub in enumerate(subs):
    evaluate_model(base_models_1[i], f"BaseModel_1{sub}")
    evaluate_model(base_models_2[i], f"BaseModel_2{sub}")
evaluate_model(meta_learner_1, "MetaLearner_1")
evaluate_model(meta_learner_2, "MetaLearner_2")
evaluate_model(meta_meta_learner, "MetaMetaLearner")


2025-06-03 22:54:04.015278: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748984044.035498   18429 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748984044.041671   18429 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748984044.057122   18429 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748984044.057148   18429 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748984044.057150   18429 computation_placer.cc:177] computation placer alr


Building dataframe using real data...
